# STEP 1:

Use www.coordenadas-gps to identify the address associated to each latitude-longitude

In [ ]:
from selenium import webdriver  # Import the Selenium web driver
from selenium.webdriver.common.keys import Keys  # Import keys module to emulate keypresses
from selenium.webdriver.support.ui import WebDriverWait  # Import WebDriverWait for waiting for elements
from selenium.webdriver.support import expected_conditions as EC  # Import expected_conditions module for defining conditions
from selenium.webdriver.common.by import By  # Import By module for locating elements
import time  # Import time module for time-related functions
from datetime import datetime  # Import datetime module for working with dates and times

def get_address(lat, lon):
    # Use Chrome as the web browser
    browser = webdriver.Chrome()

    # Open the web page
    browser.get('https://www.coordenadas-gps.com/')

    # Wait until the consent element is clickable
    wait = WebDriverWait(browser, 10)
    element = wait.until(EC.element_to_be_clickable((By.XPATH, "//p[contains(text(), 'Consentir')]")))

    # Click on the consent element
    element.click()

    # Locate the address input field
    address_input = wait.until(EC.presence_of_element_located((By.ID, 'address')))
    browser.execute_script("arguments[0].scrollIntoView(true);", address_input)
    time.sleep(5)

    # Find latitude and longitude input fields and enter values
    latitude_input = wait.until(EC.presence_of_element_located((By.ID, 'latitude')))
    latitude_input.clear()
    latitude_input.send_keys(lat)

    longitude_input = wait.until(EC.presence_of_element_located((By.ID, 'longitude')))
    longitude_input.clear()
    longitude_input.send_keys(lon)

    # Execute JavaScript to click on the "Obtener Dirección" button
    button = browser.find_element(By.XPATH, "//button[contains(text(), 'Obtener Dirección')]")
    button.click()

    time.sleep(5)
    
    # Wait for the address to load
    address_input = wait.until(EC.presence_of_element_located((By.ID, 'address')))

    # Extract information from the address field
    address_value = address_input.get_attribute('value')

    # Print the address to the console
    print("Address:", address_value)

    # Close the browser
    browser.quit()
    time.sleep(5)

    return address_value

### We add to DadosNodos a new column with the address

In [85]:
import pandas as pd

df = pd.read_csv('DatosNodos.csv', sep=';')

df['Address'] = ['']*len(df)

print(df)

     codnode     latitude     longitude  ZIPcod  IdDeliveryZone Address
0     121342  40,34569931  -3,826030016   28921              83        
1    1018226  40,37994003  -3,696700096   28041             128        
2     996618  40,41889191  -3,689340115   28001             199        
3     173838  40,50439453  -3,702736378   28049             238        
4     129378  40,32183075  -3,714220047   28903              66        
..       ...          ...           ...     ...             ...     ...
195   817045  40,35905838  -3,831180096   28922             158        
196  1021947  40,27172899  -3,699045203   28906             135        
197  1046367  40,34178162  -3,847450018   28922             147        
198   147016  40,34624863  -3,825429916   28921              83        
199   142383  40,37435913  -3,621340036   28031             107        

[200 rows x 6 columns]


In [98]:
try:
    for i, loc in df.iterrows():
        lat = loc.latitude.replace(',', '.')
        lon = loc.longitude.replace(',', '.')
        print(lat, lon)
        address = get_address(lat=lat, lon=lon)
        df.at[i, "Address"] = address
except:
    print('Última i=', i)
    hora_actual = datetime.now().strftime("%H-%M-%S")
    df.to_csv(hora_actual + '.csv', index=False, sep=';')

40.34569931 -3.826030016
Dirección: Dia, Calle Espada, 28921 Alcorcón, España
40.37994003 -3.696700096
Dirección: Avenida de Córdoba, 33, 28026 Madrid, España
40.41889191 -3.689340115
Dirección: Calle de Valenzuela, 7, 28014 Madrid, España
40.50439453 -3.702736378
Dirección: Calle del Monasterio de Montesclaros, 9, 28049 Madrid, España
40.32183075 -3.714220047
Dirección: Avenida de Rigoberta Menchú, 28093 Getafe, España
40.42784882 -3.688610077
Dirección: Calle de Ayala, 6, 28001 Madrid, España
40.46403885 -3.806499958
Dirección: Calle de Rafael Botí, 28334 Madrid, España
40.45497131 -3.480230093
Dirección: Paseo de la Estación, 4, 28850 Torrejón de Ardoz, España
40.4251709 -3.700750113
Dirección: Calle de Fuencarral, 74, 28004 Madrid, España
40.38685989 -3.757009983
Dirección: Calle de Camarena, 290, 28047 Madrid, España
40.5120697 -3.67027998
Dirección: Calle de Frómista, 28050 Madrid, España
40.37929916 -3.729460001
Dirección: Calle Carrero Juan Ramón, 28025 Madrid, España
40.440078

We save the dataframe as 'Address.csv'

In [101]:
df.to_csv('Adress.csv', index=False, sep=';', encoding='latin')

# STEP 2:
## Manual STEP

In some cases the last web page indentify directly the bussiness. We create a new column named 'Valid' that takes the value TRUE if there is the bussiness in the address.

# STEP 3:

/www.google.es/maps/ude

In [7]:
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from bs4 import BeautifulSoup

def get_address_from_maps(address):

    # Utilizamos Firefox como navegador
    browser = webdriver.Chrome()

    # Abrimos la página web
    browser.get('https://www.google.es/maps/')

    # Esperamos hasta que los elementos estén presentes en el DOM
    wait = WebDriverWait(browser, 10)

    # Esperamos hasta que el elemento esté presente y sea cliclable
    element = wait.until(EC.element_to_be_clickable((By.XPATH,'//button[@aria-label="Rechazar todo"]')))

    # Hacemos clic en el elemento
    element.click()


    address_input = wait.until(EC.presence_of_element_located((By.ID, 'searchboxinput')))
    address_input.clear()
    address_input.send_keys(address)

    # Ejecutamos un script de JavaScript para hacer clic en el botón "Obtener Dirección"
    button = wait.until(EC.element_to_be_clickable((By.ID, "searchbox-searchbutton")))
    button.click()

    time.sleep(5)

    try:
        # Esperar hasta que aparezca el elemento con "role" igual a "feed"
        arial = 'Resultados de ' + address
        elemento_feed = WebDriverWait(browser, 3).until(
            EC.presence_of_element_located((By.XPATH, f'//div[@aria-label="{arial}"]'))
        )
        # Encontrar el primer enlace dentro del elemento "feed" y hacer clic en él
        primer_enlace = elemento_feed.find_element(By.TAG_NAME, 'a')
        primer_enlace.click()

    except Exception as e:
        print('\t sin seleccion de ubicacion')

    time.sleep(5)
    try:
        div_class = EC.presence_of_element_located((By.XPATH, '//div[@class="m6QErb"]'))
        div_sin_clase = wait.until(EC.presence_of_element_located((By.XPATH, '//div[not(@class) and div[contains(@class, "Nv2PK")]]')))
        # Imprime el contenido del div encontrado
        content_html = div_sin_clase.get_attribute("outerHTML")

        soup = BeautifulSoup(content_html, 'html.parser')

        # Encontrar el div que contiene el nombre
        content = soup.find('div', class_='qBF1Pd fontHeadlineSmall').text
        business = True
    except NoSuchElementException:
        content ="NADA ENCONTRADO"
        business = False
    except TimeoutException:
        content ="Timeout"
        business = False

    # Cerramos el navegador
    browser.quit()
    time.sleep(2)
    print(business, '-->', content)

    return business, content

In [151]:
df_ad = pd.read_csv('Address.csv', sep=';', encoding='latin1')

df_ad['Business'] = ['']*len(df_ad)

print(df_ad)

     codnode       latitude       longitude  ZIPcod  IdDeliveryZone  \
0     121342  4,034,569,931  -3,826,030,016   28921              83   
1    1018226  4,037,994,003  -3,696,700,096   28041             128   
2     996618  4,041,889,191  -3,689,340,115   28001             199   
3     173838  4,050,439,453  -3,702,736,378   28049             238   
4     129378  4,032,183,075  -3,714,220,047   28903              66   
..       ...            ...             ...     ...             ...   
195   817045  4,035,905,838  -3,831,180,096   28922             158   
196  1021947  4,027,172,899  -3,699,045,203   28906             135   
197  1046367  4,034,178,162  -3,847,450,018   28922             147   
198   147016  4,034,624,863  -3,825,429,916   28921              83   
199   142383  4,037,435,913  -3,621,340,036   28031             107   

                                               Address Valid Business  
0            Dia, Calle Espada, 28921 Alcorcón, España  True           
1  

In [152]:
try:
    for i, loc in df_ad.iterrows():
        print('----> i=', i)
        if loc.Valid == True:
            continue
        address = loc.Address
        print(address)
        valid, business_name = get_address_from_maps(address)
        if valid:
            df_ad.at[i, "Valid"] = 'TRUE'
            df_ad.at[i, "Business"] = business_name
        else:
            df_ad.at[i, "Valid"] = 'FALSE'
            df_ad.at[i, "Business"] = '-'
except:
    print('Last i=', i)
    df_ad.to_csv('BusinessName.csv', index=False, sep=';')

----> i= 0
----> i= 1
Avenida de Córdoba, 33, 28026 Madrid, España
	 sin seleccion de ubicacion
True --> TH Asesores


C:\Users\ctff0\AppData\Local\Temp\ipykernel_15900\1100192901.py:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'TH Asesores' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_ad.at[i, "Bussiness"] = business_name


----> i= 2
Calle de Valenzuela, 7, 28014 Madrid, España
	 sin seleccion de ubicacion
True --> Barracuda Mx
----> i= 3
Calle del Monasterio de Montesclaros, 9, 28049 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 4
Avenida de Rigoberta Menchú, 28093 Getafe, España
True --> MAPFRE
----> i= 5
Calle de Ayala, 6, 28001 Madrid, España
	 sin seleccion de ubicacion
True --> Restaurante Ten Con Ten
----> i= 6
Calle de Rafael Botí, 28334 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 7
Paseo de la Estación, 4, 28850 Torrejón de Ardoz, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 8
Calle de Fuencarral, 74, 28004 Madrid, España
	 sin seleccion de ubicacion
True --> Superchulo Fuencarral
----> i= 9
Calle de Camarena, 290, 28047 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 10
Calle de Frómista, 28050 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 11
Calle Carrero Juan Ramón, 28025 Madr

In [153]:
try:
    for i, loc in df_ad.iterrows():
        if i < 149:
            continue
        print('----> i=', i)
        if loc.Valid == True:
            continue
        address = loc.Address
        print(address)
        valid, business_name = get_address_from_maps(address)
        if valid:
            df_ad.at[i, "Valid"] = 'TRUE'
            df_ad.at[i, "Business"] = business_name
        else:
            df_ad.at[i, "Valid"] = 'FALSE'
            df_ad.at[i, "Business"] = '-'
except:
    print('Última i=', i)
    df_ad.to_csv('BusinessName.csv', index=False, sep=';')

----> i= 149
Novotel Campo de las Naciones, Calle de Ámsterdam, 28043 Madrid, España
	 sin seleccion de ubicacion
True --> Gourmet Bar
----> i= 150
Calle de Alcalá, 292, 28027 Madrid, España
	 sin seleccion de ubicacion
True --> Parroquia Nuestra Señora Del Rosario De Fátima
----> i= 151
Avenida de Villaviciosa, 21, 28291 Alcorcón, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 152
----> i= 153
----> i= 154
Calle de María Teresa Sáenz de Heredia, 15, 28017 Madrid, España
	 sin seleccion de ubicacion
True --> Fibrosis madrid postoperatorio
----> i= 155
Calle de Ayala, 28, 28001 Madrid, España
	 sin seleccion de ubicacion
True --> Casa Dani
----> i= 156
Calle de Alcalá, 103, 28009 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 157
----> i= 158
----> i= 159
Lizarrán, Avenida de Felipe II, 28, 28009 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 160
----> i= 161
----> i= 162
----> i= 163
Calle Ruiz de Alarcón, 18, 28014 Madrid

In [155]:
df_ad.to_csv('BusinessName.csv', index=False, sep=';', encoding='latin')

# NEW DATA

We have to discard some points, so we decide to repeate the valid process with new points

In [8]:
import pandas as pd
df_new = pd.read_csv('newDatosNodos.csv', sep=';')

df_new['Address'] = ['']*len(df_new)

print(df_new)

try:
    for i, loc in df_new.iterrows():
        print('----> i=', i)
        lat = str(loc.latitude).replace(',', '.')
        lon = str(loc.longitude).replace(',', '.')
        print(lat, lon)
        address = get_address(lat=lat, lon=lon)
        df_new.at[i, "Address"] = address
except:
    print('Last i=', i)
    df_new.to_csv('newAddress.csv', index=False, sep=';', encoding='latin')

    codnode   latitude  longitude  ZIPcod  IdDeliveryZone Address
0    132469  40.426109   -3.69669   28004             277        
1    913213  40.419628   -3.70128   28013             197        
2    178568  40.311909   -3.72803   28902             146        
3    818376   0.040478   -3.36253   28804             236        
4    861012  40.238800   -3.77027   28981              80        
..      ...        ...        ...     ...             ...     ...
95   149157  40.427990   -3.68627   28001              47        
96   995939  40.501080   -3.89214   28232              89        
97   609821  40.504250   -3.70707   28049             238        
98   154178  40.508629   -3.66718   28050             136        
99   158205  40.305180   -3.72991   28901             146        

[100 rows x 6 columns]
----> i= 0
40.426109313965 -3.69669008255
Dirección: Cruz Blanca, Calle de Santa Teresa, 28004 Madrid, España
----> i= 1
40.419628143311 -3.701280117035
Dirección: Calle del Caballero 

In [30]:
df_new_filtrado = df_new[df_new['latitude'] > 30]
df_new_filtrado
df_new_filtrado.to_csv('newAddress.csv', index=False, sep=';', encoding='latin')

In [31]:
df_new_ad = pd.read_csv('newAddress.csv', sep=';', encoding='latin1')

df_new_ad['Business'] = ['']*len(df_new_ad)

print(df_new_ad)

    codnode   latitude  longitude  ZIPcod  IdDeliveryZone  \
0    132469  40.426109   -3.69669   28004             277   
1    913213  40.419628   -3.70128   28013             197   
2    178568  40.311909   -3.72803   28902             146   
3    861012  40.238800   -3.77027   28981              80   
4    135019  40.414700   -3.67669   28009             116   
..      ...        ...        ...     ...             ...   
86   149157  40.427990   -3.68627   28001              47   
87   995939  40.501080   -3.89214   28232              89   
88   609821  40.504250   -3.70707   28049             238   
89   154178  40.508629   -3.66718   28050             136   
90   158205  40.305180   -3.72991   28901             146   

                                              Address Valid Business  
0   Cruz Blanca, Calle de Santa Teresa, 28004 Madr...   NaN           
1   Calle del Caballero de Gracia, 1, 28013 Madrid...   NaN           
2             Calle Madrid, 102, 28902 Getafe, España 

In [32]:
try:
    for i, loc in df_new_ad.iterrows():
        print('----> i=', i)
        if loc.Valid == True:
            continue
        address = loc.Address
        print(address)
        valid, business_name = get_address_from_maps(address)
        if valid:
            df_new_ad.at[i, "Valid"] = 'TRUE'
            df_new_ad.at[i, "Business"] = business_name
        else:
            df_new_ad.at[i, "Valid"] = 'FALSE'
            df_new_ad.at[i, "Business"] = '-'
except:
    print('Última i=', i)
    df_new_ad.to_csv('NewBusinessName.csv', index=False, sep=';', encoding='latin')
df_new_ad.to_csv('newBusinessName.csv', index=False, sep=';', encoding='latin')

----> i= 0
Cruz Blanca, Calle de Santa Teresa, 28004 Madrid, España
True --> Cruz Blanca
----> i= 1
Calle del Caballero de Gracia, 1, 28013 Madrid, España
	 sin seleccion de ubicacion
True --> Relieve del Real Oratorio
----> i= 2
Calle Madrid, 102, 28902 Getafe, España
	 sin seleccion de ubicacion
True --> Restaurante El Tiempo
----> i= 3
Calle de Hoyos, Parla, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 4
----> i= 5
----> i= 6
Edificio Cadagua, Paseo de la Castellana, 28020 Madrid, España
	 sin seleccion de ubicacion
True --> Lateral Castellana 89
----> i= 7
Gran Vía, 55, 28013 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 8
Avenida Camino de Santiago, 28050 Madrid, España
	 sin seleccion de ubicacion
False --> Timeout
----> i= 9
unnamed road, 28805 Alcalá de Henares, España
	 sin seleccion de ubicacion
True --> Pérgola
----> i= 10
----> i= 11
----> i= 12
Amichi, Calle de Praga, 28230 Las Rozas de Madrid, España
False --> Timeout
----> i= 13

In [33]:
df_new_ad.to_csv('newBusinessName.csv', index=False, sep=';', encoding='latin')